# **Problem 2**

# ***1***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering
import statsmodels.api as sm
from subprocess import call
from IPython.display import Image
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
from sklearn.tree import plot_tree
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.model_selection import GridSearchCV

In [3]:
df = pd.read_csv("/content/drive/MyDrive/ML/Class 1 - EDA/Datasets/StateData.csv")
df.head(10)

,Population,Income,Illiteracy,LifeExp,Murder,HighSchoolGrad,Frost,Area,Longitude,Latitude,Region
0,3615,3624,2.1,69.05,15.1,41.3,20,50708,-86.7509,32.5901,South
1,365,6315,1.5,69.31,11.3,66.7,152,566432,-127.2500,49.2500,West
2,2212,4530,1.8,70.55,7.8,58.1,15,113417,-111.6250,34.2192,West
3,2110,3378,1.9,70.66,10.1,39.9,65,51945,-92.2992,34.7336,South
4,21198,5114,1.1,71.71,10.3,62.6,20,156361,-119.7730,36.5341,West
5,2541,4884,0.7,72.06,6.8,63.9,166,103766,-105.5130,38.6777,West
6,3100,5348,1.1,72.48,3.1,56.0,139,4862,-72.3573,41.5928,Northeast
7,579,4809,0.9,70.06,6.2,54.6,103,1982,-74.9841,38.6777,South
8,8277,4815,1.3,70.66,10.7,52.6,11,54090,-81.6850,27.8744,South
9,4931,4091,2.0,68.54,13.9,40.6,60,58073,-83.3736,32.3329,South


## **A**

In [4]:
features = df.drop(['LifeExp','Longitude','Latitude','Region'], axis = 1)
target = df['LifeExp']

In [ ]:
LnRmod = sm.OLS(target,sm.add_constant(features, prepend=False))
res = LnRmod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                LifeExp   R-squared:                       0.736
Model:                            OLS   Adj. R-squared:                  0.692
Method:                 Least Squares   F-statistic:                     16.74
Date:                Mon, 02 Sep 2024   Prob (F-statistic):           2.53e-10
Time:                        03:29:30   Log-Likelihood:                -51.855
No. Observations:                  50   AIC:                             119.7
Df Residuals:                      42   BIC:                             135.0
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Population       5.18e-05   2.92e-05      1.

=> Murder, as CI does not include 0, and P value is equal to 0.

# **B**

In [ ]:
DTR = DecisionTreeRegressor()
DTR.fit(features,target)

#export the decision tree.
export_graphviz(DTR,out_file='tree.dot',
                    feature_names = features.columns,
                    rounded = True, proportion = False,
                   filled = True)

call(['dot', '-Tpng', 'tree.dot', '-o', 'tree1.png', '-Gdpi=600'])
Image("tree1.png")

Output hidden; open in https://colab.research.google.com to view.

Murder is the most important feature, as it is the first split and also the second and third.

## **C**

Linear Regression, as the CART model is overfitting, with each leaf has exactly 1 feature.

## **D**

Linear Regression

This is because of its straightforward coefficients and clear statiscal measusres (p-values). And the relationship between variables is more transparent.

# ***2***

## **A**

In [6]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=1)

#CART
DTR = DecisionTreeRegressor()
DTR.fit(X_train,y_train)
print('CART R-squared:', DTR.score(X_test, y_test))
#Lasso
lass = Lasso()
lass.fit(X_train,y_train)
print('Lasso R-squared:', lass.score(X_test, y_test))


CART R-squared: -1.1094904637905456
Lasso R-squared: -0.6290476013205935


Based on the R-squared score, we can see that Lasso performed better. But, both models are worse.

## **B**

In [7]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=2)

#CART
DTR = DecisionTreeRegressor()
DTR.fit(X_train,y_train)
print('CART R-squared:', DTR.score(X_test, y_test))
#Lasso
lass = Lasso()
lass.fit(X_train,y_train)
print('Lasso R-squared:', lass.score(X_test, y_test))

CART R-squared: 0.6657766280737774
Lasso R-squared: 0.526920309135839


Based on the R-squared score, we can see that CART performed better. And both models are ok.


## **C**

Small dataset so high sensitivity to the train-test split.

Apply Cross-Validation

## **D**

In [9]:
k = KFold(10,random_state=1,shuffle=True)

dtr_score = []
lasso_score = []

for train_index, test_index in k.split(features):

    #Get the training and testing sets
    X_train, X_test = features.loc[train_index], features.loc[test_index]
    y_train, y_test = target[train_index], target[test_index]

    #Train/test CART
    DTR = DecisionTreeRegressor()
    DTR.fit(X_train,y_train)
    dtr_score.append(DTR.score(X_test, y_test))

    #Train/test Lasso
    lass = Lasso()
    lass.fit(X_train,y_train)
    lasso_score.append(lass.score(X_test,y_test))

print('CART average R-squared:', np.mean(dtr_score))
print('Lasso average R-squared:', np.mean(lasso_score))

CART average R-squared: -0.5021388794944536
Lasso average R-squared: -0.22284465988120852


Poor performance for both models.

Can apply the GridSearchCV to tune the hyperparameters to reduce overfitting and train-test split sensitivity.

In [12]:
params1 = {'min_samples_leaf':[1,2,3,4,5,6,7,8,9,10]}
params2 = {'alpha':[0.001,0.01,0.1,1,10]}

dtr_score = []
lasso_score = []

DTR = DecisionTreeRegressor()
lass = Lasso()

k = KFold(10,random_state=1,shuffle=True)

for train_index, test_index in k.split(features):

    #Get the training and testing sets
    X_train, X_test = features.loc[train_index], features.loc[test_index]
    y_train, y_test = target[train_index], target[test_index]

    #Train/test CART
    clf = GridSearchCV(DTR, params1, cv=10)
    clf.fit(X_train,y_train)
    dtr_score.append(clf.best_estimator_.score(X_test,y_test))

    #Train/test Lasso
    clf_lass = GridSearchCV(lass, params2, cv=10)
    clf_lass.fit(X_train,y_train)
    lasso_score.append(clf_lass.best_estimator_.score(X_test,y_test))

print('CART average R-squared:', np.mean(dtr_score))
print('Lasso average R-squared:', np.mean(lasso_score))

CART average R-squared: -0.4805349252093067
Lasso average R-squared: -0.0238551631538655


Lasso performed better.


## **F**

We see a slight improvement from d to e due to tuning the hyperparameters. It affected more for Lasso than CART.

In [13]:
lasso_score

[0.6575081974685693,
 -0.4380930695869676,
 0.7457507113739468,
 0.3578668161554118,
 0.5397622206941342,
 -0.28684179118558517,
 -2.0192921532348054,
 -1.173059747203653,
 0.6309964374931862,
 0.7468507464871076]

In [14]:
dtr_score

[0.4501513939935715,
 -2.090745992786253,
 0.6870430828347649,
 -1.031495390268688,
 0.4877412494401556,
 -0.9722722198483886,
 -2.8223417058127094,
 -0.7403003349382917,
 0.6764482404617856,
 0.5504224248309864]